# Аугментация данных
В данном ноутбуке выполняется аугментация данных для расширения датасета.

In [1]:
%matplotlib inline
import random

import tqdm
import glob
import cv2
from matplotlib import pyplot as plt
from pathlib import Path

import albumentations as A

import os
import sys
sys.path.append('/app/src')
from markup_convertors import yolo2bb, bb2yolo

In [14]:
def visualize_bbox(img, bbox, class_name, thickness=8):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, x_max, y_max = bbox
    x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=(255, 0, 0), thickness=thickness)
    
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [24]:
people_img_path = "/app/data/interim/images/train/"
labels_img_path = "/app/data/interim/labels/train/"

people_save_path = "/app/data/augment_ssr/images/train/"
labels_save_path = "/app/data/augment_ssr/labels/train/"

Path(people_save_path).mkdir(parents=True, exist_ok=True)
Path(labels_save_path).mkdir(parents=True, exist_ok=True)

for img in tqdm.tqdm(glob.glob(people_img_path + "*")):

    image = cv2.imread(img)
    
    height, width, ch = image.shape
    
    filename = os.path.splitext(os.path.basename(img))[0]
    
    file = open(labels_img_path + filename + ".txt", "r")
    markup = []
    for line in file.readlines():
        markup.append(yolo2bb(line, width, height))
    
    bboxes = [x[1:] for x in markup]
    category_ids = [int(x[0]) for x in markup]
    category_id_to_name = {0: 'person'}
    
    transform = A.Compose(
        [
        A.HorizontalFlip(p=0.0),
        A.VerticalFlip(p=0.0),
        A.ShiftScaleRotate(p=1.0),
        A.RandomBrightnessContrast(p=0.0),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.0)
        ],
        bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
    )
    
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
    # visualize(
    #     transformed['image'],
    #     transformed['bboxes'],
    #     transformed['category_ids'],
    #     category_id_to_name,
    # )
    
    variant = "SSR"
    
    mkp2file = []
    for bbox in transformed['bboxes']:
        mkp2file.append(bb2yolo(bbox, width, height))
        
    save_name = f'{filename}_{variant}'
    with open(f'{labels_save_path}{save_name}.txt', "w") as f:
        for mkp in mkp2file:
            mkp = [str(x) for x in mkp]
            f.write("0 " + " ".join(mkp) + "\n")
            
    cv2.imwrite(f'{people_save_path}{save_name}.jpg', transformed['image'])
    

100%|██████████| 58/58 [00:28<00:00,  2.05it/s]
